# Jax Level 0 Data Owner Experience - Chapter 9 - Getting started with Jax MLPs, CNNs, and RNNs

Link to the original blog post by Robert Tjarko Lange: https://roberttlange.com/posts/2020/03/blog-post-10/

## Part 0 - Initial Setup

In [1]:
# Import the necessary libraries
import syft as sy
sy.requires(">=0.8-beta")
import jax
import jax.numpy as jnp
import numpy as np

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /


✅ The installed version of syft==0.8.1b1 matches the requirement >=0.8b0


In [2]:
# Launch the domain
node = sy.orchestra.launch(name="test-domain-1", reset=True)
domain_client = node.login(email="info@openmined.org", password="changethis")

SQLite Store Path:
!open file:///var/folders/sz/hkfsnn612hq56r7cs5rd540r0000gn/T/7bca415d13ed1ec841f0d0aede098dbb.sqlite



In [ ]:
# TODO: upload the MNIST dataset

### Go to the Data Scientist Notebook for Part 1!

## Part 2 - Reviewing and Approving requests

In [46]:
# Get messages from domain
messages = domain_client.api.services.messages.get_all()
messages

,type,id,subject,status,created_at,linked_obj
0,syft.service.message.messages.Message,011c79bc56e640e197dfdb66d4ff4404,Approval Request,MessageStatus.UNDELIVERED,2023-05-11 09:38:41,<<class 'syft.service.request.request.Request'...
1,syft.service.message.messages.Message,de442f39e3794e89bf8ac3489259825d,Approval Request,MessageStatus.UNDELIVERED,2023-05-11 09:37:43,<<class 'syft.service.request.request.Request'...
2,syft.service.message.messages.Message,7ac4215c3a5b485881dff2b18490ad43,Approval Request,MessageStatus.UNDELIVERED,2023-05-11 09:37:13,<<class 'syft.service.request.request.Request'...
3,syft.service.message.messages.Message,991252a7c7024e6fad60c9c1102962a5,Approval Request,MessageStatus.UNDELIVERED,2023-05-11 09:47:05,<<class 'syft.service.request.request.Request'...
4,syft.service.message.messages.Message,4f0edcfa7bb34eb1b54cdc234413db8c,Approval Request,MessageStatus.UNDELIVERED,2023-05-11 09:51:10,<<class 'syft.service.request.request.Request'...


NOTE: new code exeecution requests are inserted to the list of messages instead of appending.
As a result the indices of the old messages change which might be confusing for the user.
Perhaps messages could be sorted by creation timestamp?

In [4]:
def review_request(index: int):
    request = messages[index].link
    func = request.changes[0].link
    func_name = func.service_func_name
    print(func_name)
    print(func.raw_code)

def run_submitted_function(index: int):
    request = messages[index].link
    func = request.changes[0].link
    user_func = func.unsafe_function
    real_result = user_func()
    print(real_result)
    return real_result

def accept_request(index: int, real_result: any):
    request = messages[index].link
    request.approve()
    result = request.accept_by_depositing_result(real_result)
    print(result)

In [14]:
review_request(0)


func_grad
@sy.syft_function(input_policy=sy.ExactMatch(dummy=ptr),
                  output_policy=sy.SingleExecutionExactOutput())
def func_grad(dummy):
    # Note: using different naming conventions for numpy and jax
    # compared to the original blog post, i.e. onp => np, np => jnp.
    import numpy as np
    import jax.numpy as jnp
    from jax import random, jit, grad

    # NOTE: using the same ReLU function as in the previous example
    def ReLU(x):
        """ Rectified Linear Unit (ReLU) activation function """
        return jnp.maximum(0, x)
    
    def FiniteDiffGrad(x):
        """ Compute the finite difference derivative approx for the ReLU"""
        return np.array((ReLU(x + 1e-3) - ReLU(x - 1e-3)) / (2 * 1e-3))
    
    # NOTE: generating the random matrix again.
    # Any way to share code between syft.functions?
    key = random.PRNGKey(1)
    x = random.uniform(key, (1000, 1000))

    # Compare the Jax gradient with a finite difference approximation
    print("Ja

In [18]:
result = run_submitted_function(0)


In [19]:
accept_request(0, result)

message='Request 4fa6610a7a034987b40fb0e664830e39 changes applied'


In [ ]:
# Repeat for all the remaining requests
# review_request(1)
# result = run_submitted_function(1)
# accept_request(1, result)

### Go to the Data Scientist Notebook for Part 3!